In [13]:
import pandas as pd
import numpy as np

maha_pt = pd.read_csv('combined_maha_pt.csv', header=0)
len(maha_pt.index)

2624

In [6]:
def count_words(sen):
    sen_list = sen.split(',')
    return len(sen_list)

In [7]:
maha_pt['countWords'] = maha_pt['sent'].apply(count_words)

In [8]:
workingdf = maha_pt[maha_pt['countWords'] > 3 ]
workingdf.head()

,sentID,sent,words,lexids,countWords
0,mbh12_26165.xml,"n1s,i,x6s,n1s,iic,m3s,n1s,n1s,fut a3s","etat,ca,te,toyam,vaqavAmuKa,saYjYitena,pIyamAn...","0,0,0,0,pn, ctp6,ctp3,adj ppr. ps.,0,0",9
1,mbh03_14648.xml,"n2s,i,m1s,m1s,n5s,n5s,iic,i,i,ipf[1] a3s,i,pre...","etat,SrutvA,sa,kacCapaH,tasmAt,sarasaH,ut,TAya...","0,abs,0,0,0,0,preverb,abs,preverb,0,0,0,0,0,0,0",16
3,mbh12_26044.xml,"m2p,m1s,prf a3s,m1s,m1s,n2s,pre[4] m3s,m1s","tAn,brahma,uvAca,fziH,BArgavaH,tapaH,tapyate,d...","0,pn,0,adj,adj,0,0,pn",8
4,mbh03_14642.xml,"m1s,x2p,ipf[2] a3s,pre[2] a3s,i,i,i,n7s,m1s,n1...","sa,naH,abravIt,asti,Kalu,iha,eva,sarasi,akUpar...","0,0,0,0,0,0,0,0,pn,0,0,preverb,0",13
5,mbh03_14557.xml,"m3p,x1s,ipv[2] a2s,m6s,m1s,m1s,x6s,m1d,i,x6s,m...","tEH,tvam,yAhi,kzatriyasya,eza,vAhaH,mama,vAmyO...","0,0,0,0,0,0,0,0,0,0,0,0,0",13


In [12]:
maha_pt_with_verb = pd.read_csv('maha_pt_with_noun_and_verb_count.csv', header=0)
maha_pt_with_verb.head()
len(maha_pt_with_verb.index)

2226

In [18]:
workingdf2 = pd.merge(workingdf, maha_pt_with_verb, left_on='sentID', right_on='sentID', how = 'left')
workingdf2.head()
workingdf2 = workingdf2[['sentID', 'words_x', 'sent_x', 'lexids']]
workingdf2.columns = ['sentID', 'words', 'sent', 'lexids']

In [19]:
def process_sen(sen):
    sen_list = sen.split(',')
    return sen_list

In [20]:
workingdf2['lexids'] = workingdf2['lexids'].apply(process_sen)
workingdf2['words'] = workingdf2['words'].apply(process_sen)

In [21]:
all_lexids = []
for x in workingdf2.lexids:
        all_lexids = all_lexids + x
unique_lexids = set(all_lexids)
unique_lexids.remove('')
unique_lexids.remove('0')
len(unique_lexids)
print(unique_lexids)

{'adj ppr. [1] md.', ' cab', 'gerundive_tavya', 'adj_comp', 'ctp_na', 'noun_action', 'cau ppf_actn', 'adj_ctp_upapada', 'noun_ctp_dvigu', 'pron_p1', 'ppr. [2] md.', 'adj_ctp_prAdi', 'gati', 'prap[1]', 'noun_pn', 'ppa.', 'ppr. [1] md.', 'pfp', 'pcl_rel', ' ctp4', ' adj ctp3', 'vt8am', 'indefinite adj', 'prap', 'num_cardinal', 'cdvigu', 'adj pfp. [1]', 'vt6am', 'ppr. ps.', 'noun_ck', ' ctp upapada', ' ctp3', 'pfp.', 'ca. pfp. [3]', 'ctp6', 'pp. ctp5', 'pap', 'noun_gerundive_ya', ' adj cab', ' adj cbv', 'adv', 'prap_desid', 'adj ctp2', 'adv_rel', 'vt9am', 'noun_pn_ctp_dvigu', 'ctp upapada', 'iic.', 'gerundive_tavya_cau', 'vt1m 6am', 'adj_ctp_gati', 'cvi', 'gerundive_ya', 'pfap', 'vt3am', 'vt6a', 'adj ppr. [9] ac.', 'pron_int', ' ck', 'noun_pn_ctp3', 'adv_dem', 'adj ppr. [5] ac.', 'noun_ctp_upapada', 'adj ck', 'prmp[10]', 'ctp7', 'pron_dem', '\xa0preverb', 'noun_pn_cbv', 'ca. pfp. [1]', 'adj_cdi', 'adj cbv', 'ppr. [1] ac.', 'vt4m', 'pcl_int', 'int. ppr. md.', ' ctp5', 'ctp2', 'pcl_kp', 'ad

In [22]:
master_lexid = []

for lexid in unique_lexids:
        templist = []
        i = 0
        for row in workingdf2.lexids:
            if lexid in row:
                templist.append(i)
            i += 1
        tempdf = workingdf2.iloc[templist,:]
        tempdf.sample(frac=1)
        tempdf = tempdf.reset_index(drop=True)
        try:
            if len(tempdf.index) > 1:
                master_lexid.append([lexid, tempdf.sentID[0], tempdf.words[0][tempdf.lexids[0].index(lexid)], tempdf.words[0]])
                master_lexid.append([lexid, tempdf.sentID[1], tempdf.words[1][tempdf.lexids[1].index(lexid)], tempdf.words[1]])
            else:
                master_lexid.append([lexid, tempdf.sentID[0], tempdf.words[0][tempdf.lexids[0].index(lexid)], tempdf.words[0]])
        except Exception:
                print(lexid)
df = pd.DataFrame(master_lexid, columns=['lexid','filename', 'word', 'sentence'])

ppa.
vt9am
ca. pfp. [1]
 ctp prAdi
 cd
ca. pp.
vt7am


In [23]:
df.to_csv("lexids_no_verb.csv", index=False, encoding='utf-8')